In [1]:
from abc import ABC, abstractmethod
from keras import backend as K
from keras.layers import Dense,Input,Lambda,regularizers,BatchNormalization,initializers,Activation
from keras.optimizers import adam
from keras.models import Model,load_model
import numpy as np
import random
from collections import namedtuple, deque
import sys
import pandas as pd
from task import Task
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [22]:
class Base(ABC):
    def __init__(self,in_size,out_size,act_min,act_max,lr=0.001,nl=2,activation=['relu','relu','sigmoid'],num_nodes
            =[400,300],tau=0.001,kr=None):
        self.in_size = in_size
        self.out_size = out_size
        self.act_min = act_min
        self.act_max = act_max
        self.nl=nl
        self.lr = lr
        self.num_nodes = num_nodes
        self.activations = activation
        self.kernel_regularizer = kr
        self.Create_network()
    #@abstractmethod
    def Create_network(self):
        pass
    #@abstractmethod
    def train(self):
        pass
    def Update(self):
        pass

In [24]:
?Dense

In [23]:
class Actor(Base):
    def Create_network(self):
        """Build an actor (policy) network that maps states -> actions."""

        net = layers.Dense(units=400, kernel_regularizer=layers.regularizers.l2(1e-6))(states)
        
        net = layers.Dense(units=300, kernel_regularizer=layers.regularizers.l2(1e-6))(net)

        # Add final output layer with weights initialised to Uniform[-3e-3, 3e-3]
        #raw_actions = layers.Dense(units=self.action_size, activation='sigmoid', name='raw_actions')(net)
        raw_actions = layers.Dense(units=self.action_size, activation='sigmoid', kernel_initializer = layers.initializers.RandomUniform(minval=-0.003, maxval=0.003), name='raw_actions')(net)

        # Scale [0, 1] output for each action dimension to proper range
        actions = layers.Lambda(lambda x: (x * self.action_range) + self.action_low, name='actions')(raw_actions)

        # Create Keras model
        self.model = models.Model(inputs=states, outputs=actions)

        # Define loss function using action value (Q value) gradients
        action_gradients = layers.Input(shape=(self.action_size,))
        loss = K.mean(-action_gradients * actions)

        # Incorporate any additional losses here (e.g. from regularizers)

        # Define optimizer and training function
        optimizer = optimizers.Adam(lr=self.learning_rate)
        updates_op = optimizer.get_updates(params=self.model.trainable_weights, loss=loss)
        self.train_fn = K.function(
            inputs=[self.model.input, action_gradients, K.learning_phase()],
            outputs=[],
            updates=updates_op)
        
        
        x = Input((self.in_size,))
        out=None
        for i in range(self.nl):
            if out==None:
                out=Dense(self.num_nodes[i])(x)
            else:
                out=Dense(self.num_nodes[i])(out)
            if(self.BN):
                out=BatchNormalization()(out)
            out = Activation(self.activations[i])
        return Model(inputs=[inp],outputs=[out]),inp

class Critic(Base):
    def __init__(self,in_size,out_size,act_min,act_max,lr=0.001,nl=2,activation=['relu','relu','sigmoid'],num_nodes=[400,300],tau=0.001,actin_size):
        super().__init__(in_size,out_size,act_min,act_max,lr,nl,activation,num_nodes,tau)
        self.actin_size= actin_size

IndentationError: expected an indented block (<ipython-input-23-e0c516e377bd>, line 5)

In [19]:
ss = Actor(18,4,0,900)
ss = Critic(18,4,1,0,1)

blabla
[400, 300]


In [4]:
x = Actor()

TypeError: Can't instantiate abstract class Actor with abstract methods train

In [19]:
from keras import layers, models, optimizers
from keras import backend as K
import numpy as np

class Actor:
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, action_low, action_high):
        """Initialize parameters and build model.

        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            action_low (array): Min value of each action dimension
            action_high (array): Max value of each action dimension
        """
        self.state_size = state_size
        self.action_size = action_size
        self.action_low = action_low
        self.action_high = action_high
        self.action_range = self.action_high - self.action_low

        # Initialize any other variables here
        self.learning_rate = 0.0001

        self.build_model()

    def build_model(self):


In [20]:
from keras import layers, models, optimizers
from keras import backend as K
import numpy as np

class Critic:
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size):
        """Initialize parameters and build model.

        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        
        # Initialize any other variables here
        self.learning_rate = 0.001

        self.build_model()

    def build_model(self):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        # Define input layers
        states = layers.Input(shape=(self.state_size,), name='states')
        actions = layers.Input(shape=(self.action_size,), name='actions')

        # Add hidden layer(s) for state pathway
        #net_states = layers.Dense(units=32, activation='relu')(states)
        #net_states = layers.Dense(units=64, activation='relu')(net_states)

        net_states = layers.Dense(units=400, kernel_regularizer=layers.regularizers.l2(1e-6))(states)
        net_states = layers.BatchNormalization()(net_states)
        net_states = layers.Activation('relu')(net_states)

        # Add hidden layer(s) for action pathway
        net_states = layers.Dense(units=300, kernel_regularizer=layers.regularizers.l2(1e-6))(net_states)
        net_actions = layers.Dense(units=300, kernel_regularizer=layers.regularizers.l2(1e-6))(actions)

        # Try different layer sizes, activations, add batch normalization, regularizers, etc.

        # Combine state and action pathways
        net = layers.Add()([net_states, net_actions])
        net = layers.Activation('relu')(net)

        # Add more layers to the combined network if needed

        # Add final output layer to produce action values (Q values)
        #Q_values = layers.Dense(units=1, name='q_values')(net)
        Q_values = layers.Dense(units=1, name='q_values', kernel_initializer = layers.initializers.RandomUniform(minval=-0.003, maxval=0.003))(net)

        # Create Keras model
        self.model = models.Model(inputs=[states, actions], outputs=Q_values)

        # Define optimizer and compile model for training with built-in loss function
        optimizer = optimizers.Adam(lr=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='mse')

        # Compute action gradients (derivative of Q values w.r.t. to actions)
        action_gradients = K.gradients(Q_values, actions)

        # Define an additional function to fetch action gradients (to be used by actor model)
        self.get_action_gradients = K.function(
            inputs=[*self.model.input, K.learning_phase()],
            outputs=action_gradients)

In [21]:
class DDPG_Agent():
    """Reinforcement Learning agent that learns using DDPG."""
    def __init__(self, task):
        self.task = task
        self.state_size = task.state_size
        self.action_size = task.action_size
        self.action_low = task.action_low
        self.action_high = task.action_high

        # Actor (Policy) Model
        self.actor_local = Actor(self.state_size, self.action_size, self.action_low, self.action_high)
        self.actor_target = Actor(self.state_size, self.action_size, self.action_low, self.action_high)

        # Critic (Value) Model
        self.critic_local = Critic(self.state_size, self.action_size)
        self.critic_target = Critic(self.state_size, self.action_size)

        # Initialize target model parameters with local model parameters
        self.critic_target.model.set_weights(self.critic_local.model.get_weights())
        self.actor_target.model.set_weights(self.actor_local.model.get_weights())

        # Noise process long-running mean / the speed of mean reversion / the volatility parameter
        self.exploration_mu = 0
        self.exploration_theta = 0.15
        #self.exploration_sigma = 0.3
        self.exploration_sigma = 0.2
        self.noise = OUNoise(self.action_size, self.exploration_mu, self.exploration_theta, self.exploration_sigma)

        # Replay memory
        self.buffer_size = 1000000
        self.batch_size = 64
        self.memory = ReplayBuffer(self.buffer_size, self.batch_size)

        # Algorithm parameters
        self.gamma = 0.99  # discount factor
        self.tau = 0.001  # for soft update of target parameters
 
    def reset_episode(self):
        self.noise.reset()
        state = self.task.reset()
        self.last_state = state
        return state

    def step(self, action, reward, next_state, done):
         # Save experience / reward
        self.memory.add(self.last_state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

        # Roll over last state and action
        self.last_state = next_state

    def act(self, state):
        """Returns actions for given state(s) as per current policy."""
        state = np.reshape(state, [-1, self.state_size])
        action = self.actor_local.model.predict(state)[0]
        return list(action + self.noise.sample())  # add some noise for exploration

    def learn(self, experiences):
        """Update policy and value parameters using given batch of experience tuples."""
        # Convert experience tuples to separate arrays for each element (states, actions, rewards, etc.)
        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.array([e.action for e in experiences if e is not None]).astype(np.float32).reshape(-1, self.action_size)
        rewards = np.array([e.reward for e in experiences if e is not None]).astype(np.float32).reshape(-1, 1)
        dones = np.array([e.done for e in experiences if e is not None]).astype(np.uint8).reshape(-1, 1)
        next_states = np.vstack([e.next_state for e in experiences if e is not None])

        # Get predicted next-state actions and Q values from target models
        #     Q_targets_next = critic_target(next_state, actor_target(next_state))
        actions_next = self.actor_target.model.predict_on_batch(next_states)
        Q_targets_next = self.critic_target.model.predict_on_batch([next_states, actions_next])

        # Compute Q targets for current states and train critic model (local)
        Q_targets = rewards + self.gamma * Q_targets_next * (1 - dones)
        self.critic_local.model.train_on_batch(x=[states, actions], y=Q_targets)

        # Train actor model (local)
        action_gradients = np.reshape(self.critic_local.get_action_gradients([states, actions, 0]), (-1, self.action_size))
        self.actor_local.train_fn([states, action_gradients, 1])  # custom training function

        # Soft-update target models
        self.soft_update(self.critic_local.model, self.critic_target.model)
        self.soft_update(self.actor_local.model, self.actor_target.model)   

    def soft_update(self, local_model, target_model):
        """Soft update model parameters."""
        local_weights = np.array(local_model.get_weights())
        target_weights = np.array(target_model.get_weights())

        assert len(local_weights) == len(target_weights), "Local and target model parameters must have the same size"

        new_weights = self.tau * local_weights + (1 - self.tau) * target_weights
        target_model.set_weights(new_weights)

In [22]:
## TODO: Train your agent here
import os
import pickle
#from tasks.task import Task

exportPath = './data/'
if not os.path.exists(exportPath):
    os.makedirs(exportPath)

# z axis is up
init_pose = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0]) 
target_pos = np.array([0.0, 0.0, 10.0])
task = Task(init_pose=init_pose, target_pos=target_pos, runtime=5.0)
#agent = PolicySearch_Agent(task) 
#agent = Basic_Agent(task) 
agent = DDPG_Agent(task)

# before training
resultsAll = []
high_score = -1000000.0
low_score = 1000000.0

# number of episodes to train
num_episodes = 750

training_results = {
    'score': [],
    }

for i_episode in range(1, num_episodes+1):
    # start a new episode
    state = agent.reset_episode() 
    score = 0

    episode_results = {
        'time': [],
        'x': [],
        'y': [],
        'z': [],
        'phi': [],
        'theta': [],
        'psi': [],
        'vx': [],
        'vy': [],
        'vz': [],
        'reward': [],
        }

    while True:
        action = agent.act(state)  
        next_state, reward, done = task.step(action)

        agent.step(action, reward, next_state, done)
        state = next_state
        score += reward
        high_score = max(high_score, score)
        low_score = min(low_score, score)

        # track the results for offline analysis
        episode_results['time'].append(task.sim.time)
        episode_results['x'].append(state[0])
        episode_results['y'].append(state[1])
        episode_results['z'].append(state[2])
        episode_results['phi'].append(state[3])
        episode_results['theta'].append(state[4])
        episode_results['psi'].append(state[5])
        episode_results['vx'].append(state[6])
        episode_results['vy'].append(state[7])
        episode_results['vz'].append(state[8])
        episode_results['reward'].append(reward)
        
        if done:
            print("\rEpisode = {:4d}, score = {:7.3f}, low score = {:7.3f}, high score = {:7.3f}".format(i_episode, score, low_score, high_score), end="")
            training_results['score'].append(score)
            break

    resultsAll.append(episode_results)

    sys.stdout.flush()

# save results for later analysis
with open("{}results0.bin".format(exportPath), 'wb') as pickleFile:
    pickle.dump(resultsAll, pickleFile)

Episode =  381, score =  74.538, low score =   6.820, high score = 579.943

KeyboardInterrupt: 

In [11]:
ch =[]
ah =[]
for i_episode in range(1, num_episodes+1):
    state = agent.Reset() # start a new episode
    episode_reward=0
    steps_count =0
    while True:
        action = agent.Get_action(state[np.newaxis,:])
        next_state, reward, done = task.step(action[0]*900)
        steps_count+=1
        episode_reward+=reward
        agent.Replay.add(state, action, reward, next_state, done)
        state = next_state
        info = agent.train_models()
        if info:
            ch.append(info[0])
            ah.append(info[1])
        if done:
            print("\rEpisode = {:4d}, reward = {:7.3f} , steps count = {}".format(
                i_episode, episode_reward,steps_count), end="")  # [debug]
            reward_results['episode'].append(i_episode)
            reward_results['reward'].append(episode_reward)
            break
    sys.stdout.flush()
    

Episode =  500, reward =  84.504 , steps count = 20

In [1]:
import os
import pickle
#from tasks.task import Task

exportPath = './data/'
if not os.path.exists(exportPath):
    os.makedirs(exportPath)

# z axis is up
init_pose = np.array([0.0, 0.0, 1.0, 0.0, 0.0, 0.0]) 
target_pos = np.array([0.0, 0.0, 10.0])
task = Task(init_pose=init_pose, target_pos=target_pos, runtime=5.0)
#agent = PolicySearch_Agent(task) 
#agent = Basic_Agent(task) 
agent = DDPG_Agent(task)

# before training
resultsAll = []
high_score = -1000000.0
low_score = 1000000.0

# number of episodes to train
num_episodes = 750

training_results = {
    'score': [],
    }

for i_episode in range(1, num_episodes+1):
    # start a new episode
    state = agent.reset_episode() 
    score = 0

    episode_results = {
        'time': [],
        'x': [],
        'y': [],
        'z': [],
        'phi': [],
        'theta': [],
        'psi': [],
        'vx': [],
        'vy': [],
        'vz': [],
        'reward': [],
        }

    while True:
        action = agent.act(state)  
        next_state, reward, done = task.step(action)

        agent.step(action, reward, next_state, done)
        state = next_state
        score += reward
        high_score = max(high_score, score)
        low_score = min(low_score, score)

        # track the results for offline analysis
        episode_results['time'].append(task.sim.time)
        episode_results['x'].append(state[0])
        episode_results['y'].append(state[1])
        episode_results['z'].append(state[2])
        episode_results['phi'].append(state[3])
        episode_results['theta'].append(state[4])
        episode_results['psi'].append(state[5])
        episode_results['vx'].append(state[6])
        episode_results['vy'].append(state[7])
        episode_results['vz'].append(state[8])
        episode_results['reward'].append(reward)
        
        if done:
            print("\rEpisode = {:4d}, score = {:7.3f}, low score = {:7.3f}, high score = {:7.3f}".format(i_episode, score, low_score, high_score), end="")
            training_results['score'].append(score)
            break

    resultsAll.append(episode_results)

    sys.stdout.flush()

# save results for later analysis
with open("{}results0.bin".format(exportPath), 'wb') as pickleFile:
    pickle.dump(resultsAll, pickleFile)

NameError: name 'np' is not defined